# 🚀 Quick Start: Customer Support Bot

A **5-minute** introduction to building a customer support agent with LangGraph and Ollama.

## Prerequisites
```bash
# 1. Install Ollama: https://ollama.ai
ollama pull llama3.1:latest  # or use any model you have available

# 2. Install packages
%pip install langgraph langchain-ollama langchain-core
```

In [22]:
%pip install langgraph langchain-ollama langchain-core

Note: you may need to restart the kernel to use updated packages.


## Step 1: Create a Simple Tool

Tools let the bot take actions.

In [23]:
from langchain_core.tools import tool

@tool
def get_return_policy() -> str:
    """Get the store's return policy."""
    return "Returns accepted within 30 days. Items must be unused in original packaging."

@tool
def track_order(order_id: str) -> str:
    """Track an order by ID."""
    return f"Order #{order_id} is in transit. Expected delivery: Nov 1, 2025."

tools = [get_return_policy, track_order]
print("✅ Tools created")

✅ Tools created


## Step 2: Connect to Ollama

Use a local LLM - no API keys needed!

In [24]:
from langchain_ollama import ChatOllama

# Using llama3.1:latest (available on your system)
llm = ChatOllama(model="llama3.1:latest", temperature=0)
llm_with_tools = llm.bind_tools(tools)

print("✅ Ollama connected with llama3.1:latest")

✅ Ollama connected with llama3.1:latest


## Step 3: Build the Graph

This creates the agent workflow.

In [26]:
from langgraph.graph import StateGraph, MessagesState, END, START
from langgraph.prebuilt import ToolNode

# Ensure llm_with_tools is available (redefine if needed)
if 'llm_with_tools' not in globals():
    from langchain_ollama import ChatOllama
    llm = ChatOllama(model="llama3.1:latest", temperature=0)
    llm_with_tools = llm.bind_tools(tools)

# Agent node: decides what to do
def agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Router: decides if we need tools
def should_continue(state: MessagesState):
    last = state["messages"][-1]
    return "tools" if last.tool_calls else END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", agent)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

graph = workflow.compile()
print("✅ Graph built")

✅ Graph built


## Step 4: Test It!

Ask questions and watch it work.

In [27]:
from langchain_core.messages import HumanMessage

# Ensure graph is available
if 'graph' not in globals():
    print("❌ Graph not found. Please run the previous cells first.")
else:
    # Test 1: Policy question
    try:
        result = graph.invoke({"messages": [HumanMessage("What's your return policy?")]})
        print("Bot:", result["messages"][-1].content)
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure you've run all previous cells in order.")

Bot: Based on the output of the tool call, I can provide an answer to the user's question.

The return policy allows for returns within 30 days, and items must be unused in their original packaging. This information should help the user understand what is required for a successful return.


In [28]:
# Test 2: Order tracking
if 'graph' in globals():
    try:
        result = graph.invoke({"messages": [HumanMessage("Track order #12345")]})
        print("Bot:", result["messages"][-1].content)
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Graph not found. Please run the previous cells first.")

Bot: The tool call response indicates that the order is in transit and has an expected delivery date of November 1, 2025. I can now provide a formatted answer to your original question:

"Your order #12345 is currently in transit and is expected to be delivered on November 1, 2025."


## Step 5: Chat Loop

Have a conversation!

In [29]:
def chat(message: str):
    """Simple chat function."""
    if 'graph' not in globals():
        print("❌ Graph not found. Please run the previous cells first.")
        return None
    
    print(f"You: {message}")
    try:
        result = graph.invoke({"messages": [HumanMessage(message)]})
        response = result["messages"][-1].content
        print(f"Bot: {response}\n")
        return response
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Try it
if 'graph' in globals():
    chat("Hi, can you help me?")
    chat("What's your return policy?")
    chat("Can you track order #99999?")
else:
    print("❌ Graph not found. Please run the previous cells first.")

You: Hi, can you help me?
Bot: You're welcome! I'd be happy to assist you further.

What seems to be the issue or question you have?

You: What's your return policy?
Bot: Based on the output of the tool call, I can provide an answer to the user's question.

The return policy allows for returns within 30 days, and items must be unused in their original packaging. This information should help the user understand what is required for a successful return.

You: Can you track order #99999?
Bot: The tool call response indicates that the order is in transit and has an expected delivery date of November 1, 2025.



## 🎉 That's It!

You've built a working customer support agent in 5 steps!

### What Just Happened?
1. **Tools** - You defined actions the bot can take
2. **LLM** - Connected to local Ollama (free!)
3. **Graph** - Created agent → tools → agent loop
4. **Chat** - The bot decides which tools to use automatically

### Next Steps
- Add more tools for your use case
- Try different Ollama models (`mistral:7b`, `qwen2.5:7b`)
- Add conversation memory
- Check out the full demo notebook: `customer_support_bot_demo.ipynb`

### Try These Models
```bash
ollama pull mistral:7b      # Better reasoning
ollama pull qwen2.5:7b      # Strong performance
ollama pull llama3.1:8b     # Larger, more capable
```

### Resources
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [Ollama Models](https://ollama.ai/library)
- Full project: See `customer-support-bot.zip`

**Happy coding! 🚀**